<a href="https://colab.research.google.com/github/yashnagrawal/image_classification_cnn_seq/blob/main/c4w1a2_tf_Sequential_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1) Importing packages

In [10]:
import tensorflow as tf
import numpy as np
import math
import h5py

In [11]:
def load_dataset():
    train_dataset = h5py.File('datasets/train_happy.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('datasets/test_happy.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes

    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))

    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [12]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

In [13]:
print("X shape: ", X_train_orig.shape)
print("Y shape: ", Y_train_orig.shape)

X shape:  (600, 64, 64, 3)
Y shape:  (1, 600)


# 3) Preprocessing input

In [14]:
X_train = X_train_orig/255.0
Y_train = Y_train_orig.T
X_test = X_test_orig/255.0
Y_test = Y_test_orig.T

In [15]:
print("X train: ", X_train.shape)
print("Y train: ", Y_train.shape)
print("X test: ", X_test.shape)
print("Y test: ", Y_test.shape)

X train:  (600, 64, 64, 3)
Y train:  (600, 1)
X test:  (150, 64, 64, 3)
Y test:  (150, 1)


# 4) Defining the model

In [16]:
def model():
  """
  ZEROPAD2D(padding 3) -> CONV2D (32 7x7 filters, stride 1) -> BATCHNORM (axis = 3) -> RELU -> MAXPOOL -> FLATTEN -> DENSE (1 unit, sigmoid activation)
  Return:
    model -- tensorflow model
  """
  model = tf.keras.Sequential()
  model.add(tf.keras.Input(shape = (64, 64, 3)))
  model.add(tf.keras.layers.ZeroPadding2D(padding = 3))
  model.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = (7, 7), strides = 1))
  model.add(tf.keras.layers.BatchNormalization(axis = 3))
  model.add(tf.keras.layers.ReLU())
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

  return model

In [17]:
happy_model = model()

happy_model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = tf.keras.losses.BinaryCrossentropy(),
    metrics = tf.keras.metrics.BinaryAccuracy()
)

In [18]:
happy_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d (ZeroPaddin  (None, 70, 70, 3)         0         
 g2D)                                                            
                                                                 
 conv2d (Conv2D)             (None, 64, 64, 32)        4736      
                                                                 
 batch_normalization (Batch  (None, 64, 64, 32)        128       
 Normalization)                                                  
                                                                 
 re_lu (ReLU)                (None, 64, 64, 32)        0         
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                                 
 dense (Dense)               (None, 1)                 1

# 5) Training the model

In [19]:
happy_model.fit(x = X_train, y = Y_train, batch_size = 16, epochs = 10)

Epoch 1/10
38/38 [==============================] - 6s 127ms/step - loss: 2.4498 - binary_accuracy: 0.7217
Epoch 2/10
38/38 [==============================] - 3s 89ms/step - loss: 0.3673 - binary_accuracy: 0.9167
Epoch 3/10
38/38 [==============================] - 4s 92ms/step - loss: 0.4797 - binary_accuracy: 0.9200
Epoch 4/10
38/38 [==============================] - 4s 119ms/step - loss: 0.3646 - binary_accuracy: 0.9167
Epoch 5/10
38/38 [==============================] - 4s 113ms/step - loss: 0.5541 - binary_accuracy: 0.9200
Epoch 6/10
38/38 [==============================] - 4s 97ms/step - loss: 0.4982 - binary_accuracy: 0.9300
Epoch 7/10
38/38 [==============================] - 3s 82ms/step - loss: 0.2033 - binary_accuracy: 0.9667
Epoch 8/10
38/38 [==============================] - 5s 120ms/step - loss: 0.1597 - binary_accuracy: 0.9683
Epoch 9/10
38/38 [==============================] - 3s 82ms/step - loss: 0.2855 - binary_accuracy: 0.9483
Epoch 10/10
38/38 [=======================

# 6) Evaluating the model

In [20]:
happy_model.evaluate(x = X_test, y = Y_test)

5/5 [==============================] - 1s 78ms/step - loss: 9.9571 - binary_accuracy: 0.4600


[9.95712947845459, 0.46000000834465027]